In [1]:
from sage.all import matrix, vector, SR, I
import time
import scipy.sparse as sp
import numpy as np
import scipy.sparse.linalg as spla

totaltime0 = time.time()

# We denote by CBF and RBF our rings of Complex Ball Fields and Real Ball Fields (which 
# are equipped with interval arithmetics over the real and complex respectivley)
CBF = ComplexBallField(200)
RBF = RealBallField(200)



# --- Global Variables ---
# We will look at a triangle of frequencies with j1>j2>= 0 and j1+j2 < SUM_J_LIMIT (set I_t in Proof of Lemma 4.6)
# Recall also that j1+j2 must be odd. 
# The code assumes SUM_J_LIMIT is even
SUM_J_LIMIT = 160
if (SUM_J_LIMIT % 2) != 0:
    print("Warning, the code is designed to work only for SUM_J_LIMIT even")
IDX_SIZE = 0        # IDX_SIZE will be the number of frequencies on I_t
coord_to_idx = {}   # To each pair of frequencies (j, k) in I_t we will associate a index: (j, k) -> idx
idx_to_coord = []   # The vector mapping idx -> (j, k)

# We start creating our maps sending frequencies to indices and viceversa
current_idx = 0
for j1 in range(1, SUM_J_LIMIT):
    for j2 in range(0, j1): # j1 > j2
        if (((j1 + j2) % 2) != 0) & (j1+j2 < SUM_J_LIMIT):  # We check j1+j2 odd and <SUM_J_LIMIT
            # We add (j1, j2) to our correspondances
            coord = (j1, j2)
            coord_to_idx[coord] = current_idx
            idx_to_coord.append(coord)
            current_idx += 1
    
IDX_SIZE = len(idx_to_coord)
print(f"Mappings [index <-> pairs of frequencies] completed, total basis elements: N = {IDX_SIZE}")

def get_idx(j1, j2):
    res = coord_to_idx.get((j1, j2), -1)
    if res == -1: 
        print(f"WARNING: Trying to access wrong index!")
        return None
    return res

def get_coord(idx):
    if 0 <= idx < IDX_SIZE:
        return idx_to_coord[idx]
    print(f"WARNING: Your index is not in [0, IDX_SIZE)")
    return None



# We compute H, J, T. We also compute W to be the vector corresponding to the weights in frequency space
# That is W{i] is the squared L^2 norm of the element of the basis with index i, so that the inner product <u, v>_(L2)
# can be simply computed as the sum in i of u[i] v[i] W[i]
# RING denotes the RING we are using. For example, RING = ComplexBallField for complex interval arithmetic balls, 
# RING = SR for symbolic computation (fractions), or CDF for complex double numbers (which are not rigorous)
# c is either 'U' (unstable), 'S' (stable) or 'T' (total). Hu + Hs = H

# Since H is a diagonal operator, we implement it just as a vector (encoding the diagonal)
def build_H_vector(RING, c):
    H = vector(RING, IDX_SIZE)

    # Recall from (4.13) that Hu has zero entries except -8/5 at the Eu entry
    Hu = vector(RING, IDX_SIZE)
    for i in range(IDX_SIZE): Hu[i] = 0
    Hu[get_idx(1, 0)] = -RING(8)/RING(5)
    if c == 'U': return Hu

    # Now, we compute the full H via (4.14)
    for i in range(IDX_SIZE):
        i1, i2 = get_coord(i)
        H[i] = 1 - (RING(2))/(RING(i1**2 + i2**2))
    if c == 'S': return H - Hu
    elif c == 'T': return H
    raise ValueError("Invalid case c. Must be 'S', 'U', or 'T'.")


# W[i] just contains the L^2 norm squared of the i-th element of the basis

def build_W_vector(RING):
    W = vector(RING, IDX_SIZE)
    val = 2 * RING(pi)**2
    for i in range(IDX_SIZE): 
        coord = get_coord(i)
        if coord[1] == 0:
            W[i] = 2 * val
        else:
            W[i] = val
    return W

            
# We compute J according to Lemma 4.3 in the paper. Since j + k is odd, we never have j + k == SUM_J_LIMIT
# Thus, the only case where our matrix does not rigorously compute J E_{j, k} is when j + k == SUM_J_LIMIT - 1 
# in which case there would be a component on E_{j+1, k+1} outside of our basis

def build_J_matrix(RING):
    entries = {}
    for i in range(IDX_SIZE):
        j, k = get_coord(i)

        # Case 1
        if (j, k) == (1, 0):
            entries[get_idx(1, 0), i] = RING(I)/RING(2)
            entries[get_idx(2, 1), i] = -RING(1)/RING(2)

        # Case 2
        elif k == 0:
            entries[get_idx(j-1, 1), i] = RING(j)/RING(2)
            if j != SUM_J_LIMIT-1:
                entries[get_idx(j+1, 1), i] = -RING(j)/RING(2) 

        # Cases 3 and 4 agree always except for (j-1, k+1)
        else:
            entries[get_idx(j-1, k-1), i] = RING(j-k) / RING(4) 
            entries[get_idx(j+1, k-1), i] = -RING(j+k) / RING(4) 
            if j == k+1: 
                entries[i, i] = RING(I) * RING(j+k) * RING((-1)**(k)) / RING(4)
            else:
                entries[get_idx(j-1, k+1), i] = RING(j+k) / RING(4)
            if j+k != SUM_J_LIMIT-1:
                entries[get_idx(j+1, k+1), i] = -RING(j-k) / RING(4)
                
    J = matrix(RING, IDX_SIZE, IDX_SIZE, entries, sparse=True)
    return J


# We simply construct T according to T = Hs^(1/2) * J * Hs^(1/2) as specified in Theorem 1.5
# Similarly to J, it can compute T E_{j, k} rigorously (provided RING is an interval-arithemtics ring)
# as long as we do not exit our basis. That corresponds to the case j+k = SUM_J_LIMIT - 1
def build_T_matrix(RING):
    H_sta = build_H_vector(RING, 'S')
    J = build_J_matrix(RING)
    H_sta_sqrt = vector(RING, IDX_SIZE)
    for i in range(IDX_SIZE):
        H_sta_sqrt[i] = H_sta[i]**(1/2)
    return diagonal_matrix(H_sta_sqrt) * J * diagonal_matrix(H_sta_sqrt)


# We define globally the matrix & vectors that we will use. CBF = ComplexBallField will be the interval-arithmetic library from Sage that we will use
# In the case of H, and the weight of our inner product, we can use RBF = RealBallField, since they are real.
T_cbf = build_T_matrix(CBF)
Hs_vec = build_H_vector(RBF, 'S')
Hu_vec = build_H_vector(RBF, 'U')
weight = build_W_vector(RBF)


""" We also want to create the matrix T with scipy sparse. This is a numerical matrix that will not give rigorous results. 
It allows to compute a numerical guess of (T-lambda)^(-1) v [or its adjoint]. Then, the error is rigorously
bounded via Lemma 4.7. See also the proof of Lemma 4.6 for more details about this"""


def sage_sparse_to_scipy(sage_mat):
    # We will convert complex double to scipy with a dictionary {(row, col): val}
    # from sage complex double in order to avoid running through the N^2 indices of our matrix
    # This is significantly faster since the matrix T is sparse (only contains entries with "neighbouring" frequencies for input and output)
    d = sage_mat.dict()
    if d:
        rows, cols = zip(*d.keys())
        data = [complex(v) for v in d.values()]
    else:
        rows, cols, data = [], [], []

    return sp.csc_matrix((data, (rows, cols)), 
                         shape=sage_mat.dimensions(), 
                         dtype=complex)
    

T_cdf = build_T_matrix(CDF) # We first build the numerical matrix with complex double field in sage
T_scipy = sage_sparse_to_scipy(T_cdf) # Then, we convert it to a sparse scipy matrix

sqrt_weight = np.sqrt(np.array([float(w.mid()) for w in weight])) # We also create a numerical version of sqrt(weight)

print("All matrices computed")

Mappings [index <-> pairs of frequencies] completed, total basis elements: N = 3240
All matrices computed


In [2]:
def solve_trusted_least_squares(lambda_cbf, b, trusted_indices, is_dual=False):
    """ Solves min || (T - lambda*I) * v - b || using SciPy.
    If is_dual=True, solves for (T - lambda)* = -T - lambda.conj() (since T is skew-symmetric) """
    
    # First, we obtain numerical values for lambda and b (which are CBF), and define define A 
    # (the matrix for which we want to minimize ||Av - b||) depending on is_dual    
    lambda_val = complex(lambda_cbf)
    b_np = np.array([complex(x) for x in b], dtype=np.complex128)
    N = T_scipy.shape[0]   
    if not is_dual:
        A = T_scipy - lambda_val * sp.eye(N, format='csc')
    else:
        A = -T_scipy - lambda_val.conjugate() * sp.eye(N, format='csc')

    # We restrict our system to trusted_indices and solve using least squares scipy method
    A_trusted = (sp.diags(sqrt_weight) @ A)[:, trusted_indices]
    solution = spla.lsqr(A_trusted, b_np * sqrt_weight)
    v_trusted_data = solution[0]

    # We reconstruct our numerical guess filling with 0 outside trusted indices and convert it 
    # to CBF so that further computations done with our guess are rigorous.
    v_full_np = np.zeros(N, dtype=np.complex128)
    v_full_np[trusted_indices] = v_trusted_data
    return vector(CBF, v_full_np)
    
def compute_primal_dual_rigorous(v_cbf, v_star_cbf, trusted_indices, lambda_cbf):
    """Rigorously computes <(T-lambda)^{-1} e, e*> using the Primal-Dual Bound method (Lemma 4.7)"""
    
    # We get a Numerical Approximation (non-rigorous) of (A-lam)^(-1) e and (A-lam)^(-T) e*
    u0_cbf = solve_trusted_least_squares(lambda_cbf, v_cbf, trusted_indices, False)
    u0_star_cbf = solve_trusted_least_squares(lambda_cbf, v_star_cbf, trusted_indices, True)

    r_cbf = v_cbf - (T_cbf * u0_cbf - lambda_cbf * u0_cbf)
    # Notice that (T-lambda)^* = -T-lambda since T is skew
    r_star_cbf = v_star_cbf + (T_cbf * u0_star_cbf + lambda_cbf.conjugate() * u0_star_cbf)

    # Compute the main terms of the primal-dual
    res_main = CBF(0)
    for i in range(IDX_SIZE):
        res_main += u0_cbf[i] * v_star_cbf[i].conjugate() * weight[i]
        res_main += r_cbf[i] * u0_star_cbf[i].conjugate() * weight[i]
    
    # We compute the norms of the residuals
    norm_r_sq_rbf = RBF(0)
    norm_r_star_sq_rbf = RBF(0)
    for i in range(IDX_SIZE):
        norm_r_sq_rbf += r_cbf[i].abs()**2 * weight[i]
        norm_r_star_sq_rbf += r_star_cbf[i].abs()**2 * weight[i]
    norm_r_rbf = norm_r_sq_rbf.sqrt()
    norm_r_star_rbf = norm_r_star_sq_rbf.sqrt()
    
    # The final error is given by Lemma 4.7
    error_radius = (norm_r_rbf * norm_r_star_rbf) / abs(lambda_cbf.real())


    # This function returns a complex ball res_main and a real ball error_radius.
    # We rigorously know that the result of <A^(-1) v, v*> is within distance < error_radius from some element of res_main
    # One could incorporate the error_radius in res_main by using res_main.add_error(error_radius)
    # However, it is convenient to not do it since CBF are balls on each real and imaginary component (hence, complex squares)
    # Thus, we would be creating a square big enough to include our circle getting unnecessarily bigger regions
    return res_main, error_radius

def Phi(lambda_cbf):
    """ This function takes CBF intervals for lambda and outputs a rigorous enclosure for Phi(lambda) """
    
    # Define a trusted region. Since J propagates to neighbours which increase j+k at most by 2
    # we can ensure a rigorous computation of T v for any v supported on modes (j, k) with 
    # j+k < SUM_J_LIMIT -2 (since T v will be supported on j+k < SUM_J_LIMIT)
    trusted_indices = []
    for i in range(IDX_SIZE):
        j, k = get_coord(i)
        if j+k < SUM_J_LIMIT - 2:
            trusted_indices.append(i)

    # We define the data to compute Phi. We also need to compute <(T-lambda)^(-1) v, Eu> with v = T * E_u
    idx_10 = get_idx(1, 0)
    E_u = vector(CBF, IDX_SIZE)
    for i in range(IDX_SIZE): E_u[i] = CBF(0)
    E_u[idx_10] = CBF(1)
    v = T_cbf * E_u
    resolvent_main, resolvent_error = compute_primal_dual_rigorous(v, E_u, trusted_indices, lambda_cbf)

    # Finally, we compute Phi(lambda) via formula (4.15)
    Phi_result = CBF(1) - CBF(8)/CBF(3) * resolvent_main / weight[idx_10]
    Phi_error = RBF(8)/RBF(3) * resolvent_error / weight[idx_10]
    return Phi_result, Phi_error
 


#The following code snippet is not necessary for the proof. It allows to evaluate Phi at a point
#time0 = time.time()
#Phi_main, err_bound = Phi(CBF(0.10000 - 0.5875 * I).add_error(2e-3))
#time1 = time.time()
#print("TOTAL COMPUTATION TIME FOR PHI = ", time1 - time0)

#print("Phi(lambda) = ", Phi_main, err_bound)

#if Phi_main.abs() > err_bound:
#    print("We are in the good regime")
#else:
#    print("We are in the BAD regime")
#    print(abs(Phi_main), err_bound)

#signal = Phi_main.mid().abs()
#error1 = err_bound.upper()
#error2 = Phi_main.rad()
#print("Signal / error ratio = ", Phi_main.mid().abs() / (error1 + error2))

In [3]:
# We define the contour corners from (4.16) in the paper
# Then, we define a sequence of points along our contour
corners = [10/100 - 57/100 * I, 17/100 - 57/100 * I, 17/100 - 63/100 * I, 10/100 - 63/100 * I]
contour_points = []
for i in range(4):
    vec = corners[(i+1)%4] - corners[i%4]
    # We choose the points so that they are at distance approx 1e-2. 
    Nsteps = round(abs(vec) / 1e-2) 
    step = vec / Nsteps
    for k in range(Nsteps):
        contour_points.append(CBF(corners[i%4] + k * step))
CP_SIZE = len(contour_points)

# In order to define a set of CBF covering our contour, we use CBF([p_curr, p_next]) being p_curr and p_next
# two of the points selected over our contour. That new CBF ball is ensured to contain both p_curr and p_next, and any point of the segment in between
# In particular Gamma[i] satisfy the covering and non-disjointness hypothesis of Lemma 4.9
Gamma = vector(CBF, CP_SIZE)
for i in range(CP_SIZE):
    p_curr = contour_points[i]
    p_next = contour_points[(i+1)%CP_SIZE]
    Gamma[i] = CBF([p_curr, p_next])

# Now, we verify hypothesis (1) of Lemma 4.9.
success = True
theta = vector(RBF, CP_SIZE+1)
direction = vector(RBF, CP_SIZE)
time0 = time.time()

for i in range(CP_SIZE):
    print(" I am going to perform the ", i, "-th evaluation on Phi")
    res, err = Phi(Gamma[i])

    # Now, we proceed to compute the candidate for theta[j]. Since hypothesis (1) requires Re( Phi(Gamma[j]) * exp(-i theta[j]) ) > 0 
    # the reasonable candidate is to take theta[j] to be the polar angle of the center of Phi(Gamma[j])
    # Also, interval arithmetics is not needed to compute theta[j], since they are just "candidates". Hypothesis (1) of Lemma 4.7 will be then rigorously checked
    center = res.mid()
    if abs(center.real()) < 1e-6:
        center += 2e-6
    slope = center.imag() / center.real()
    if center.real() < 0:
        theta[i] = RBF(arctan(slope) + pi)
    else:
        theta[i] = RBF(arctan(slope))

    # Now, we proceed to rigorously verify the hypothesis (1) of Lemma 4.9 with our "guessed" values of theta[i]
    # Notice that exp(-I * theta[j]) has absolute value 1 so does not modify the size err
    check = res * exp(-I * theta[i])
    if check.real() - err > 0:
        pass
    else:
        success = False
        print("Hypothesis failed on ball number :", i)
    
# Regarding hypothesis (1), we can take theta[CP_SIZE] = theta[0] since Gamma[0] = Gamma[CP_SIZE]
theta[CP_SIZE] = theta[0]
if success:
    time1 = time.time()
    print("Succesfull winding number computation! Time computing Phi(Gamma_i) = ", time1-time0)

# Lastly, we need to verify hypothesis (2). In order to do so, we use Remark 4.10. As long as hypothesis (1) is satisfied
# we can always modify our vector theta[i] by adding 2*pi*n[i] so that hypothesis (2) is satisfied. In order to do so
# we define the following function of (angle1, angle2), which returns angle2' = angle2 + 2pi n such that | angle1 - angle2' | < pi
def close_angle(angle1, angle2):
    aux = (angle2 - angle1 + RBF(pi)) / (2 * RBF(pi))
    guess = int(floor(aux.lower()))
    if guess < aux.lower() and aux.upper() < guess+1:
        return angle2 - 2 * RBF(pi) * guess
    print("Error, you called close_angle with parameters: ", angle1, angle2)

# We enforce condition (2) on Lemma 4.9 via Remark 4.10 with the function close_angle
for i in range(1, CP_SIZE+1):
    theta[i] = close_angle(theta[i-1], theta[i])

# Finally, we apply Lemma 4.9 to compute the number of roots of Phi(lambda) that we enclosed in contour
print("Final winding number = ", theta[CP_SIZE] - theta[0])
print("Number of roots of Phi enclosed = ", (theta[CP_SIZE] - theta[0]) / (-2 * RBF(pi)))


totaltime1 = time.time()
print("Total time of our computations : ", totaltime1 - totaltime0)

 I am going to perform the  0 -th evaluation on Phi
 I am going to perform the  1 -th evaluation on Phi
 I am going to perform the  2 -th evaluation on Phi
 I am going to perform the  3 -th evaluation on Phi
 I am going to perform the  4 -th evaluation on Phi
 I am going to perform the  5 -th evaluation on Phi
 I am going to perform the  6 -th evaluation on Phi
 I am going to perform the  7 -th evaluation on Phi
 I am going to perform the  8 -th evaluation on Phi
 I am going to perform the  9 -th evaluation on Phi
 I am going to perform the  10 -th evaluation on Phi
 I am going to perform the  11 -th evaluation on Phi
 I am going to perform the  12 -th evaluation on Phi
 I am going to perform the  13 -th evaluation on Phi
 I am going to perform the  14 -th evaluation on Phi
 I am going to perform the  15 -th evaluation on Phi
 I am going to perform the  16 -th evaluation on Phi
 I am going to perform the  17 -th evaluation on Phi
 I am going to perform the  18 -th evaluation on Phi
 I 